In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
from tensorflow.keras.models import Model, load_model, clone_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Multiply, Add, Lambda, Concatenate, Reshape, Flatten
from tensorflow.keras.initializers import GlorotUniform, RandomUniform, Constant
from tensorflow.keras.callbacks import LambdaCallback
import jax
import jax.numpy as jnp
from jax import random, grad, jit

2024-12-03 17:21:51.672805: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-03 17:21:51.688648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733242911.707064  726620 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733242911.712670  726620 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 17:21:51.731497: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Verificar si hay GPUs disponibles
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow está utilizando la GPU")
else:
    print("TensorFlow no está utilizando la GPU")

TensorFlow no está utilizando la GPU


In [3]:
# Cargar los módulos preentrenados (unit_module y carry_module)
unit_addition_model = load_model('unit_addition_module.keras')
unit_carry_model = load_model('unit_carry_module.keras')
unit_addition_model.trainable = False
unit_carry_model.trainable = False
unit_addition_model.name = 'unit_addition_model'
unit_carry_model.name = 'unit_carry_model'

In [4]:
# Función para generar los datos
def generate_final_data():
    x_data = []
    y_data = []
    for a_dec in range(10):
        for a_unit in range(10):
            for b_dec in range(10):
                for b_unit in range(10):
                    x_data.append([a_dec, a_unit, b_dec, b_unit])  # Entrada
                    sum_units = (a_unit + b_unit) % 10
                    carry_units = 1 if (a_unit + b_unit) >= 10 else 0
                    sum_dec = (a_dec + b_dec + carry_units) % 10
                    carry_dec = 1 if (a_dec + b_dec + carry_units) >= 10 else 0
                    y_data.append([carry_dec, sum_dec, sum_units])  # Salida
    return jnp.array(x_data), jnp.array(y_data)

# Función para crear parámetros entrenables (v_0, ..., v_11)
def init_params(epsilon=0):
    w_values_init = jnp.array([0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0], dtype=jnp.float32)
    u_values_init = jnp.array([0, 1, 0, 0], dtype=jnp.float32)
    v_values_init = jnp.array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1], dtype=jnp.float32)
    key = random.PRNGKey(0)
    keys = random.split(key, 16 + 4 + 12)  # Total 32 claves necesarias
    w_params = {f'w{i}': random.normal(keys[i], (1,)) * epsilon + w_values_init[i] for i in range(16)}
    u_params = {f'u{i}': random.normal(keys[16 + i], (1,)) * epsilon + u_values_init[i] for i in range(4)}
    v_params = {f'v{i}': random.normal(keys[20 + i], (1,)) * epsilon + v_values_init[i] for i in range(12)}
    params = {**w_params, **u_params, **v_params}
    return params

# Modelo dinámico en JAX
def model(params, x):
    # Extraer unidades y decenas de los valores de entrada
    unit_input_1 = (params['w0'] * x[0]) + (params['w1'] * x[1]) + (params['w2'] * x[2]) + (params['w3'] * x[3])
    unit_input_2 = (params['w4'] * x[0]) + (params['w5'] * x[1]) + (params['w6'] * x[2]) + (params['w7'] * x[3])
    
    # Llamar a los modelos unit_module y carry_module
    units_input = jnp.array([unit_input_1, unit_input_2])
    units_input = units_input[None, None, :]
    unit_output = jnp.array(unit_addition_model(units_input))  # Salida para unidades
    unit_carry_output = jnp.array(unit_carry_model(units_input))  # Salida de acarreo de unidades

    # Tomar el valor máximo de las predicciones (argmax en JAX)
    unit_val = jnp.argmax(unit_output, axis=-1)
    carry_unit_val = jnp.argmax(unit_carry_output, axis=-1) 
    
    dec_input_1 = (params['w8'] * x[0]) + (params['w9'] * x[1]) + (params['w10'] * x[2]) + (params['w11'] * x[3]) + (params['u0'] * unit_val[0]) + (params['u1'] * carry_unit_val[0])
    dec_input_2 = (params['w12'] * x[0]) + (params['w13'] * x[1]) + (params['w14'] * x[2]) + (params['w15'] * x[3]) + (params['u2'] * unit_val[0]) + (params['u3'] * carry_unit_val[0])

    # Llamar a los modelos unit_module y carry_module
    decs_input = jnp.array([dec_input_1, dec_input_2])
    decs_input = decs_input[None, None, :]
    dec_output = jnp.array(dec_addition_model(decs_input))  # Salida para decenas
    dec_carry_output = jnp.array(dec_carry_model(decs_input))  # Salida de acarreo de decenas

    # Tomar el valor máximo de las predicciones (argmax en JAX)    
    dec_val = jnp.argmax(dec_output, axis=-1)
    carry_dec_val = jnp.argmax(dec_carry_output, axis=-1)

    # Calcular las salidas combinadas con los parámetros v
    salida_1 = (params['v0'] * carry_dec_val) + (params['v1'] * dec_val) + (params['v2'] * carry_unit_val) + (params['v3'] * unit_val)
    salida_2 = (params['v4'] * carry_dec_val) + (params['v5'] * dec_val) + (params['v6'] * carry_unit_val) + (params['v7'] * unit_val)
    salida_3 = (params['v8'] * carry_dec_val) + (params['v9'] * dec_val) + (params['v10'] * carry_unit_val) + (params['v11'] * unit_val)

    return salida_1, salida_2, salida_3

# Función de pérdida
def loss_fn(params, x, y):
    y_pred_1, y_pred_2, y_pred_3 = model(params, x)
    return jnp.mean((y_pred_1 - y[0]) ** 2) + jnp.mean((y_pred_2 - y[1]) ** 2) + jnp.mean((y_pred_3 - y[2]) ** 2)
    
# Función para entrenar el modelo
def update_params(params, x, y, lr):
    # Asegúrate de usar JAX para los gradientes y operaciones
    gradients = grad(loss_fn)(params, x, y)
    step_loss = loss_fn(params, x, y)
    new_params = jax.tree.map(lambda p, g: p - lr * g, params, gradients)
    return new_params, step_loss


def train_model(params, x_train, y_train, lr=0.01, epochs=100):
    final_loss = 0
    # Convertir x_train y y_train a arrays de JAX (si aún no lo son)
    x_train = jnp.array(x_train)
    y_train = jnp.array(y_train)
    
    # Entrenar el modelo
    for epoch in range(epochs):  # Número de épocas
        params, step_loss = update_params(params, x_train[epoch], y_train[epoch], lr)
        final_loss += step_loss
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {step_loss}")
        
    final_loss = final_loss / epochs
    return params, final_loss

# Función para imprimir las predicciones y el loss en cada época
def print_predictions_and_loss(epoch, predictions, y_train):
    pred_count = 0
    total_examples = x_train.shape[0]
    
    for i in range(total_examples):
        # Obtener las predicciones para la unidad, decena y acarreo
        normalized_pred = [int(jnp.round(predictions[j][i])) for j in range(3)]
        
        # Concatenar las predicciones en un número de 3 dígitos
        concatenated_pred = int("".join(str(pred) for pred in normalized_pred))
        
        # Generar la salida esperada, concatenando los valores reales de y_train
        expected_output = int("".join(str(int(round(val))) for val in y_train[i]))
        
        # Comprobar si la predicción es igual a la salida esperada
        if concatenated_pred == expected_output:
            pred_count += 1

    print(f"Epoch {epoch + 1}:")
    print(f"Predicciones correctas: {pred_count} de {total_examples}")
    print("-" * 40)

    # Si todas las predicciones son correctas, detener el entrenamiento
    if pred_count == total_examples:
        print("¡Todas las combinaciones han sido aprendidas correctamente! Deteniendo entrenamiento.")
        return True
    return False

def predictions(params, x_train, y_train):
    pred_count = 0
    total_examples = x_train.shape[0]   
    
    for i in range(total_examples):
        prediction = model(params, x_train[i])
        # Obtener las predicciones para la unidad, decena y acarreo
        normalized_pred = [int(jnp.round(prediction[j].item())) for j in range(3)]
        
        # Concatenar las predicciones en un número de 3 dígitos
        concatenated_pred = int("".join(str(pred) for pred in normalized_pred))
        
        # Generar la salida esperada, concatenando los valores reales de y_train
        expected_output = int("".join(str(int(round(val))) for val in y_train[i]))
        
        # Comprobar si la predicción es igual a la salida esperada
        if concatenated_pred == expected_output:
            pred_count += 1

    print(f"Predicciones correctas: {pred_count} de {total_examples}")

    if pred_count == total_examples:
        print("¡Todas las combinaciones han sido aprendidas correctamente!")

def count_predictions(params, x_train, y_train):
    pred_count = 0
    total_examples =  x_train.shape[0]   
    
    for i in range(total_examples):
        prediction = model(params, x_train[i])
        # Obtener las predicciones para la unidad, decena y acarreo
        normalized_pred = [int(jnp.round(prediction[j].item())) for j in range(3)]
        
        if normalized_pred[0] == y_train[i,0]:
            if normalized_pred[1] == y_train[i,1]:
                if normalized_pred[2] == y_train[i,2]:
                    pred_count += 1

        if (normalized_pred[0] != y_train[i,0]) or (normalized_pred[1]!= y_train[i,1]) or (normalized_pred[2] != y_train[i,2]):
            print(f'Error en la suma: {x_train[i]}')
            break
            
    print(f"Predicciones correctas: {pred_count} de {total_examples}")

    if pred_count == total_examples:
        print("¡Todas las combinaciones han sido aprendidas correctamente!")

In [15]:
x_train, y_train = generate_final_data()
params = init_params()

count_predictions(params, x_train, y_train)

ValueError: All input arrays must have the same shape.

In [16]:
x = [0, 1, 9, 9]
y_train = [1, 0, 0]
params = init_params()

units_input = jnp.array([x[1], x[3]])
units_input = units_input[None, None, :]

# Llamar a los modelos unit_module y carry_module
unit_output = jnp.array(unit_addition_model(units_input))  # Salida para unidades
unit_carry_output = jnp.array(unit_carry_model(units_input))  # Salida de acarreo de unidades
# Tomar el valor máximo de las predicciones (argmax en JAX)
unit_val = jnp.argmax(unit_output, axis=-1)
carry_unit_val = jnp.argmax(unit_carry_output, axis=-1)

decs_input = jnp.array([x[0], x[2], carry_unit_val[0]])
decs_input = decs_input[None, None, :]

dec_output = jnp.array(dec_addition_model(decs_input))  # Salida para decenas
dec_carry_output = jnp.array(dec_carry_model(decs_input))  # Salida de acarreo de decenas

dec_val = jnp.argmax(dec_output, axis=-1)
carry_dec_val = jnp.argmax(dec_carry_output, axis=-1)
# Calcular las salidas combinadas con los parámetros v
salida_1 = (params['v0'] * carry_dec_val) + (params['v1'] * dec_val) + (params['v2'] * carry_unit_val) + (params['v3'] * unit_val)
salida_2 = (params['v4'] * carry_dec_val) + (params['v5'] * dec_val) + (params['v6'] * carry_unit_val) + (params['v7'] * unit_val)
salida_3 = (params['v8'] * carry_dec_val) + (params['v9'] * dec_val) + (params['v10'] * carry_unit_val) + (params['v11'] * unit_val)

print(salida_1, salida_2, salida_3)

[1.] [0.] [0.]


In [ ]:
x_train, y_train = generate_final_data()
params = init_params()
x=x_train[0]
print(jnp.array([x[1], x[3]]))
new_params, final_loss = train_model(params, x_train, y_train, lr=0.01, epochs=100)
print(final_loss)

# Hacer predicciones después del entrenamiento
predictions = model(params, x_train)
print("Predicciones:", predictions)

In [33]:
model = build_dynamic_model()
model.summary()

for var in model.trainable_variables:
    if var.name == "v_values:0":
        print("Valores iniciales de v_values:", var.numpy())


# Predicciones
total_examples = x_train.shape[0]
pred_count = 0

# Realizar las predicciones para todos los ejemplos
predictions = model.predict(x_train)

# Si `predictions` contiene múltiples arrays (uno por salida del modelo):
if isinstance(predictions, list):
    # Concatenamos las predicciones en columnas
    predictions_df = pd.DataFrame(
        {f"Salida_{i+1}": pred.flatten() for i, pred in enumerate(predictions)}
    )
else:
    # Si `predictions` es un solo array
    predictions_df = pd.DataFrame(predictions)

# Guardar como archivo CSV
predictions_df.to_csv("predicciones_completas.csv", index=False)

# Inicializar listas para almacenar las predicciones y las salidas reales
predicted_values = []
expected_values = []

for i in range(total_examples):
    # Normalizar y redondear la predicción (cada salida del modelo)
    normalized_pred = [
        np.round(predictions[0][i]).astype(int),  # Primer valor del primer array
        np.round(predictions[1][i]).astype(int),  # Primer valor del segundo array
        np.round(predictions[2][i]).astype(int)   # Primer valor del tercer array
    ]
    
    # Concatenar los tres elementos en normalized_pred como un solo número
    concatenated_pred = int("".join(str(pred) for pred in normalized_pred))

    # Comparar la predicción con la salida real
    expected_output = int("".join(str(int(round(val))) for val in y_train[i]))  # Convertir la salida esperada en un número
    
    # Almacenar en las listas
    predicted_values.append(concatenated_pred)
    expected_values.append(expected_output)
    
    if concatenated_pred == expected_output:
        pred_count += 1    

print(f'Predicciones correctas: {pred_count} de {total_examples}.')

# Crear un DataFrame con los datos
data = {
    "Predicción": predicted_values,
    "Valor Esperado": expected_values
}
df = pd.DataFrame(data)

# Guardar como archivo CSV
df.to_csv("predicciones.csv", index=False)

print(f"Archivo 'predicciones.csv' guardado con éxito.")

AttributeError: Exception encountered when calling Multiply.call().

[1m'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'sparse'[0m

Arguments received by Multiply.call():
  • args=(['tf.Tensor(shape=(1,), dtype=float32)', '<KerasTensor shape=(None,), dtype=float32, sparse=False, name=keras_tensor_22>'],)
  • kwargs=<class 'inspect._empty'>